In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LogisticRegression 
from sklearn.datasets import make_hastie_10_2
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
# from xgboost import XGBClassifier

In [5]:
data = pd.read_csv("final_selected_variables.csv")

In [6]:
data.shape

(96397, 23)

In [7]:
data.drop(columns=["Unnamed: 0"])

,Merch_description_part_count0__max,Merchnum_count0__max,Merchnum_count1__max,Merchnum_count1__sum,Merchnum_count3__max,cardnum-merch_description_part_count14__max,cardnum-merch_description_part_count14__sum,cardnum-merch_description_part_count30__max,cardnum-merch_description_part_count3__sum,cardnum-merch_state_count14__max,...,cardnum-merch_state_merch_zip_count3__sum,cardnum-merch_zip_count14__max,cardnum-merchnum_count14__max,cardnum-merchnum_count14__sum,cardnum-merchnum_count1__max,cardnum-merchnum_count30__max,cardnum-merchnum_count3__max,cardnum-merchnum_count3__sum,Recnum,Fraud
0,-0.510270,-0.497168,-0.507695,-0.252697,-0.542314,-0.461148,-0.187507,-0.482323,-0.155707,-0.504417,...,-0.156323,-0.463631,-0.456162,-0.183189,-0.424413,-0.476666,-0.431276,-0.153873,1,0
1,-0.482932,-0.469230,-0.483051,-0.246306,-0.519227,-0.434575,-0.180833,-0.456436,-0.148848,-0.481059,...,-0.149486,-0.437273,-0.429515,-0.176517,-0.396893,-0.450565,-0.403872,-0.147028,2,0
2,-0.338311,-0.321432,-0.352675,-0.212496,-0.397089,-0.293996,-0.145525,-0.319485,-0.112567,-0.357485,...,-0.113311,-0.297837,-0.288542,-0.141221,-0.251301,-0.312484,-0.258899,-0.110819,3,0
3,-0.510270,-0.497168,-0.507695,-0.251864,-0.542314,-0.461148,-0.187507,-0.482323,-0.155707,-0.504417,...,-0.156323,-0.463631,-0.456162,-0.183189,-0.424413,-0.476666,-0.431276,-0.153873,4,0
4,-0.510270,-0.497168,-0.507695,-0.251032,-0.542314,-0.461148,-0.186638,-0.482323,-0.154814,-0.504417,...,-0.155433,-0.463631,-0.456162,-0.182320,-0.424413,-0.476666,-0.431276,-0.152981,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96392,-0.400813,-0.415596,-0.435739,-0.234037,-0.474904,-0.383560,-0.168020,-0.406738,-0.135682,-0.436215,...,-0.136358,-0.386674,-0.378357,-0.163708,-0.344059,-0.400457,-0.351263,-0.133888,96749,0
96393,-0.397056,-0.381468,-0.405634,-0.226230,-0.446702,0.108768,-0.015858,0.072885,-0.127304,0.656265,...,-0.128005,0.101654,0.115351,-0.011596,-0.310440,0.083123,-0.317787,-0.125527,96750,0
96394,1.932733,1.999503,1.694653,1.075426,1.520883,-0.117094,-0.101094,-0.147148,-0.066910,-0.201983,...,-0.067790,-0.122372,-0.111143,-0.096805,-0.068091,-0.138725,-0.076468,-0.065255,96751,0
96395,1.651543,1.712136,1.539137,1.061711,1.375193,1.640237,0.340277,1.564832,0.386637,1.342760,...,0.384416,1.620676,1.651114,0.344421,1.751899,1.587379,1.735789,0.387380,96752,0


In [8]:
train_test = data[ (data.Recnum <=84299) & (data.Recnum >= 3338)]
ott = data[data.Recnum >84299]

In [9]:
train_test.shape

(80639, 23)

In [19]:
x = train_test.copy().drop(columns = ["Fraud","Recnum"])
y = train_test["Fraud"]
oot_x = ott.copy().drop(columns = ["Fraud","Recnum"])
oot_y = ott["Fraud"]

In [20]:
# # 使用imlbearn库中上采样方法中的SMOTE接口
# from imblearn.over_sampling import SMOTE
# # 定义SMOTE模型，random_state相当于随机数种子的作用
# smo = SMOTE(random_state=42,sampling_strategy = 'minority')
# x, y = smo.fit_sample(x, y)

In [21]:
x.shape

(80639, 21)

In [22]:
# calculate FDR rate 
def FDR(x,y,model):
    # make the prediction result a probality not a label, so we can get the predicted fraud distribution
    pred = model.predict_proba(x)[:,1]
#     pred =model.predict(train_x)
    y_df = pd.DataFrame(y)
    y_df['pred'] = pred
    top = int(len(y_df) * 0.03)
    numbads = sum(y_df['Fraud'] == 1)
    fdr = y_df.sort_values(by = 'pred', ascending = False).head(top).Fraud.sum()/numbads
    return fdr

In [26]:
# test model 
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.3)
            

d = 3
l = 0.01
e = 300

gb =GradientBoostingClassifier(max_depth=d, learning_rate=l, n_estimators=e)
gb.fit(train_x, train_y)

In [25]:
FDR(train_x,train_y,gb)

In [42]:
%%time
gb_FDR = []

maxdepth = [3,4,5,6]
learningrate = [0.1, 0.01, 0.001]
estimators = [100,300,500,800]

# maxdepth = [3]
# learningrate = [0.1]
# estimators = [100]
for d in maxdepth:
    for l in learningrate:
        for e in estimators:
            FDR_train_tot = 0
            FDR_test_tot = 0
            FDR_oot_tot = 0
            for i in range(10):
                train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.3)

                gb =GradientBoostingClassifier(max_depth=d, learning_rate=l, n_estimators=e)
                gb.fit(train_x, train_y)
               
                # FDR train
                FDR_train =  FDR(train_x,train_y,gb)
                FDR_train_tot = FDR_train + FDR_train_tot
                FDR_train_avg = FDR_train_tot/(i+1)
#                 print 

                # FDR test
                FDR_test = FDR(test_x,test_y,gb)
                FDR_test_tot = FDR_test + FDR_test_tot
                FDR_test_avg = FDR_test_tot/(i+1)

                # FDR oot
                FDR_oot = FDR(oot_x,oot_y,gb)
                FDR_oot_tot = FDR_oot + FDR_oot_tot            
                FDR_oot_avg = FDR_oot_tot/(i+1)

                print (FDR_train,FDR_test,FDR_oot,FDR_train_avg,FDR_test_avg,FDR_oot_avg)
                gb_FDR.append([e, d, l, FDR_train,FDR_test,FDR_oot,FDR_train_avg,FDR_test_avg,FDR_oot_avg])

0.9690721649484536 0.8741258741258742 0.5083798882681564 0.9690721649484536 0.8741258741258742 0.5083798882681564
0.9784768211920529 0.8371212121212122 0.3687150837988827 0.9737744930702532 0.8556235431235432 0.43854748603351956
0.9770491803278688 0.9186046511627907 0.46368715083798884 0.9748660554894584 0.8766172458032924 0.44692737430167595
0.9620462046204621 0.8969465648854962 0.48044692737430167 0.9716610927722094 0.8816995755738433 0.4553072625698324
0.9770491803278688 0.9069767441860465 0.5027932960893855 0.9727387102833414 0.8867550092962839 0.4648044692737431
0.9756888168557536 0.8844621513944223 0.4860335195530726 0.9732303947120767 0.8863728663126403 0.468342644320298
0.965 0.8880597014925373 0.553072625698324 0.9720546240389228 0.8866138427669112 0.48044692737430167
0.9679595278246206 0.8945454545454545 0.4301675977653631 0.9715427370121351 0.8876052942392292 0.47416201117318435
0.9708265802269044 0.896414342629482 0.36312849162011174 0.9714631640359984 0.8885840773937017 0.

1.0 0.8949416342412452 0.36312849162011174 0.9997198879551821 0.889160210840496 0.329608938547486
1.0 0.8857142857142857 0.3743016759776536 0.9997599039615847 0.888667935822466 0.3359936153232242
0.9934747145187602 0.8901960784313725 0.26256983240223464 0.9989742552812316 0.8888589536485794 0.3268156424581005
1.0 0.892 0.45251396648044695 0.9990882269166504 0.8892079587987372 0.3407821229050279
0.9983739837398374 0.9011857707509882 0.3016759776536313 0.9990168025989691 0.8904057399939622 0.3368715083798882
0.8589743589743589 0.8032786885245902 0.5586592178770949 0.8589743589743589 0.8032786885245902 0.5586592178770949
0.853225806451613 0.8024193548387096 0.5195530726256983 0.8561000827129859 0.80284902168165 0.5391061452513966
0.8633440514469454 0.8292682926829268 0.5418994413407822 0.8585147389576391 0.8116554453487422 0.5400372439478583
0.8665568369028006 0.8045977011494253 0.553072625698324 0.8605252634439294 0.809891009298913 0.5432960893854748
0.8653198653198653 0.8175182481751825

0.9347471451876019 0.8588235294117647 0.3854748603351955 0.9336712772210944 0.8737866639219117 0.47905027932960886
0.9384359400998337 0.8951310861423221 0.5139664804469274 0.9346242097968422 0.8780555483659939 0.4860335195530726
0.9261083743842364 0.9305019305019305 0.4972067039106145 0.9332049038947413 0.8867966120553166 0.48789571694599626
0.9434276206322796 0.8913857677902621 0.5586592178770949 0.934665292000104 0.8874522057317373 0.4980047885075818
0.9477124183006536 0.86328125 0.48044692737430167 0.9362961827876727 0.8844308362652702 0.49581005586592175
0.9320066334991708 0.8528301886792453 0.5195530726256983 0.9358195662000612 0.8809196532001563 0.49844816883923027
0.9251247920133111 0.846441947565543 0.4581005586592179 0.9347500887813862 0.877471882636695 0.494413407821229
0.767479674796748 0.758893280632411 0.5363128491620112 0.767479674796748 0.758893280632411 0.5363128491620112
0.7516778523489933 0.75 0.5642458100558659 0.7595787635728706 0.7544466403162056 0.5502793296089385

0.8189233278955954 0.807843137254902 0.5307262569832403 0.800291039310422 0.7657486945640708 0.5516759776536313
0.7864238410596026 0.75 0.5251396648044693 0.7975175996602581 0.7625989556512567 0.5463687150837989
0.8105436573311368 0.7969348659003831 0.5698324022346368 0.7996886092720712 0.7683216073594443 0.5502793296089385
0.8133971291866029 0.8049792531120332 0.5307262569832403 0.8016469692598615 0.7735584138955284 0.5474860335195532
0.819672131147541 0.7635658914728682 0.5586592178770949 0.8039001144958213 0.772309348592696 0.5488826815642458
0.8163265306122449 0.8181818181818182 0.5586592178770949 0.8052808273976462 0.777406289658154 0.5499689633767846
0.8016 0.7983539094650206 0.547486033519553 0.8049127446578815 0.7795010516388406 0.5497206703910613
CPU times: user 20h 34min 40s, sys: 5min 52s, total: 20h 40min 32s
Wall time: 20h 58min 23s


In [43]:
gb_DF = pd.DataFrame(gb_FDR, columns = ['n_estimators', 'max_depth', 'learning_rate', 'train', 'test', 'oot','train_avg', 'test_avg', 'oot_avg'])
gb_DF

,n_estimators,max_depth,learning_rate,train,test,oot,train_avg,test_avg,oot_avg
0,500,3,0.100,0.969072,0.874126,0.508380,0.969072,0.874126,0.508380
1,500,3,0.100,0.978477,0.837121,0.368715,0.973774,0.855624,0.438547
2,500,3,0.100,0.977049,0.918605,0.463687,0.974866,0.876617,0.446927
3,500,3,0.100,0.962046,0.896947,0.480447,0.971661,0.881700,0.455307
4,500,3,0.100,0.977049,0.906977,0.502793,0.972739,0.886755,0.464804
...,...,...,...,...,...,...,...,...,...
235,800,6,0.001,0.810544,0.796935,0.569832,0.799689,0.768322,0.550279
236,800,6,0.001,0.813397,0.804979,0.530726,0.801647,0.773558,0.547486
237,800,6,0.001,0.819672,0.763566,0.558659,0.803900,0.772309,0.548883
238,800,6,0.001,0.816327,0.818182,0.558659,0.805281,0.777406,0.549969


In [44]:
gb_DF.to_excel('gb_FDR3.xlsx', index=True, header=True)

In [60]:
para = []

maxdepth = [3,4,5,6]
learningrate = [0.1, 0.01, 0.001]
estimators = [100,300,500,800]

# maxdepth = [3]
# learningrate = [0.1]
# estimators = [100]
for d in maxdepth:
    for l in learningrate:
        for e in estimators:
            for i in range(10):
                para.append([e, d, l])

In [61]:
para

[[100, 3, 0.1],
 [100, 3, 0.1],
 [100, 3, 0.1],
 [100, 3, 0.1],
 [100, 3, 0.1],
 [100, 3, 0.1],
 [100, 3, 0.1],
 [100, 3, 0.1],
 [100, 3, 0.1],
 [100, 3, 0.1],
 [300, 3, 0.1],
 [300, 3, 0.1],
 [300, 3, 0.1],
 [300, 3, 0.1],
 [300, 3, 0.1],
 [300, 3, 0.1],
 [300, 3, 0.1],
 [300, 3, 0.1],
 [300, 3, 0.1],
 [300, 3, 0.1],
 [500, 3, 0.1],
 [500, 3, 0.1],
 [500, 3, 0.1],
 [500, 3, 0.1],
 [500, 3, 0.1],
 [500, 3, 0.1],
 [500, 3, 0.1],
 [500, 3, 0.1],
 [500, 3, 0.1],
 [500, 3, 0.1],
 [800, 3, 0.1],
 [800, 3, 0.1],
 [800, 3, 0.1],
 [800, 3, 0.1],
 [800, 3, 0.1],
 [800, 3, 0.1],
 [800, 3, 0.1],
 [800, 3, 0.1],
 [800, 3, 0.1],
 [800, 3, 0.1],
 [100, 3, 0.01],
 [100, 3, 0.01],
 [100, 3, 0.01],
 [100, 3, 0.01],
 [100, 3, 0.01],
 [100, 3, 0.01],
 [100, 3, 0.01],
 [100, 3, 0.01],
 [100, 3, 0.01],
 [100, 3, 0.01],
 [300, 3, 0.01],
 [300, 3, 0.01],
 [300, 3, 0.01],
 [300, 3, 0.01],
 [300, 3, 0.01],
 [300, 3, 0.01],
 [300, 3, 0.01],
 [300, 3, 0.01],
 [300, 3, 0.01],
 [300, 3, 0.01],
 [500, 3, 0.01],
 [5

In [62]:
para_pd = pd.DataFrame(para, columns = ['n_estimators', 'max_depth', 'learning_rate'])



In [63]:
para_pd.to_csv("para.csv")